In [1]:
import os
import pandas as pd
from datetime import datetime
from scipy.stats import shapiro, ttest_ind

# import plotly.io as pio
# pio.renderers.default = 'notebook' 

import plotly.express as px

In [2]:
df1 = pd.read_csv('./datasets/control_group.csv', sep=';')
df2 = pd.read_csv('./datasets/test_group.csv', sep=';')

df = pd.concat([df1, df2])
df.columns = [c.lower().replace(' ', '_').replace('#_of_', '') for c in df.columns]

In [3]:
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y"))

In [4]:
df.head()

,campaign_name,date,spend_[usd],impressions,reach,website_clicks,searches,view_content,add_to_cart,purchase
0,Control Campaign,2019-08-01,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2019-08-02,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,2019-08-03,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,2019-08-04,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,2019-08-05,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['metric'] = df['website_clicks'] / df['spend_[usd]']

In [6]:
df = df[['campaign_name', 'date', 'metric']]

In [7]:
df.head()

,campaign_name,date,metric
0,Control Campaign,2019-08-01,3.077193
1,Control Campaign,2019-08-02,4.615822
2,Control Campaign,2019-08-03,2.777636
3,Control Campaign,2019-08-04,1.579897
4,Control Campaign,2019-08-05,NaN


In [8]:
df.describe()

,metric
count,59.000000
mean,2.397971
std,0.864073
min,0.963738
25%,1.705499
50%,2.328343
75%,3.045468
max,4.615822


In [9]:
fig = px.histogram(df, x='metric', nbins=10)

fig.write_image('./hist.png')

In [10]:
# is it normal??

stat, p = shapiro(df.dropna()['metric'])

print(stat)
print(p)

0.9757992029190063
0.28722497820854187


In [11]:
# is it significant??

control_group = df[df['campaign_name'] == 'Control Campaign']['metric'].dropna()
test_group = df[df['campaign_name'] == 'Test Campaign']['metric'].dropna()

stat, p = ttest_ind(control_group, test_group)

if p > .05:
    print('oof, not normal')

oof, not normal


In [12]:
fig = px.line(df, x='date', y='metric', color='campaign_name')

fig.write_image('./time.png')